In [9]:
import sys
sys.path.insert(0, "../app/chat")
import os
from conversational_rag_chain import create_conversational_rag_chain
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.chat_history import BaseChatMessageHistory
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langchain.chains.constitutional_ai.principles import PRINCIPLES
from langchain.evaluation import Criteria
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType
from langchain_openai import AzureChatOpenAI


load_dotenv()

True

In [10]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

In [11]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    return InMemoryHistory()

chain=create_conversational_rag_chain("""You are an assistant for question-answering tasks. 
                                     
You can only use the following pieces of retrieved context to answer the question. 
                                     
If you cannot answer the question with the provided context or there is no context provided, inform the user that you do not have enough information to answer the question
                                     
Use three sentences maximum and keep the answer concise.
                                     
You will have a chat history, but you must only answer the last question.
                                     
You MUST answer in dutch.
                                     
The date of today is: """ + str(datetime.now()),"CIB-lid",3,0.7, get_session_history)

In [14]:
result = chain.invoke(
        {"input": 'wat is er de voorbije jaren allemaal veranderd in de brusselse woningfiscaliteit?'},
        config={"configurable": {"session_id": "abc123"}},
    )  
answer = result['answer']
print(answer)

De nieuwe Brusselse Huisvestingscode, die in werking trad op 28 juli 2013, heeft verschillende veranderingen geïntroduceerd in de Brusselse woningfiscaliteit. Enkele belangrijke veranderingen zijn onder andere: 
- Grotere efficiëntie in de strijd tegen slechte huisvesting en de professionaliteit van diverse actoren, zoals de Directie Gewestelijke Huisvestinginspectie en sociale verhuurkantoren.
- Meer aandacht voor de sociale situatie van bewoners, waarbij het vermijden van dakloosheid belangrijk is.
- Introduceren van nieuwe woonconcepten zoals "solidair wonen" en "intergenerationeel wonen".
- Meer transparantie en samenwerking tussen verschillende diensten.
- Meer participatie door hervorming van adviesraden voor sociale huurders en de oprichting van een extern adviesorgaan voor woningtoewijzingen.


In [18]:
llm=AzureChatOpenAI(
        openai_api_version=str(os.getenv("AZURE_OPENAI_API_VERSION")),
        azure_deployment=str(os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")),
    )
evaluator = load_evaluator("labeled_criteria", llm=llm, criteria=Criteria.CORRECTNESS)

In [22]:
evaluator.evaluate_strings(
    prediction=answer,
    input="wat is er de voorbije jaren allemaal veranderd in de brusselse woningfiscaliteit?",
    reference="De Brusselse Huisvestingscode is in de afgelopen jaren gewijzigd om verschillende veranderingen door te voeren in de strijd tegen onveilige en leegstaande woningen en om hervormingen door te voeren in de toewijzing van gemeentelijke woningen."
)

{'reasoning': 'To determine if the submission meets the criteria of correctness, we will compare it to the reference statement and assess if it is correct, accurate, and factual. \n\n1. The submission states that the new Brusselse Huisvestingscode was implemented on 28 July 2013, which is consistent with the reference statement. \n2. The submission mentions that the code introduced changes in the Brusselse woningfiscaliteit to combat bad housing and improve the professionalism of various actors. This aligns with the reference statement that changes were made to address unsafe and vacant housing.\n3. The submission highlights the focus on the social situation of residents and the importance of preventing homelessness, which corresponds to the reference statement.\n4. The submission mentions the introduction of new housing concepts such as "solidair wonen" and "intergenerationeel wonen," which is not mentioned in the reference statement. It is unclear if these concepts were part of the c